In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

/home/campus.stonybrook.edu/pdutta/anaconda3/envs/stratified_subsampling/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
no_of_iteration = 2

In [3]:
INPUT_PATH= "/data/projects/shared_data/collab_data/10_transcripts/Data"
INPUT_FOLDERNAME = "tpm"
INPUT_FILENAME = "gbm_gtex.csv"
OUTPUT_PATH = "/data/projects/shared_data/collab_data/10_transcripts/Results/Empirical_p_values"
OUTPUT_FILE_NAME = INPUT_FILENAME.split('.')[0]+"_epochs_p.csv"
OUTPUT_FILE_NAME1 = INPUT_FILENAME.split('.')[0]+"_empirical_p.csv"

In [4]:
df = pd.read_csv(INPUT_PATH+"/"+INPUT_FOLDERNAME+"/"+INPUT_FILENAME, sep= " ")


In [5]:
df

,status,sample,ENST00000461347.5,ENST00000322723.8,ENST00000484328.1,ENST00000454824.5,ENST00000417652.5,ENST00000453992.5,ENST00000436894.1,ENST00000494618.1,ENST00000356936.5,ENST00000466274.1
1,gtex_brain,GTEX-1117F-3226-SM-5N9CT,3.030013,37.190473,-1.089298e-08,2.099978e-01,-1.089298e-08,-1.089298e-08,-1.089298e-08,8.020099,25.029236,1.020012
2,gtex_brain,GTEX-111FC-3126-SM-5GZZ2,6.400012,64.430756,-1.089298e-08,7.299781e-01,-1.089298e-08,1.400032e-01,-1.089298e-08,20.770018,35.399553,4.560158
3,gtex_brain,GTEX-111FC-3326-SM-5GZYV,13.230457,84.447506,-1.089298e-08,1.719944e+00,-1.089298e-08,1.200018e-01,-1.089298e-08,18.799511,38.839773,7.190038
4,gtex_brain,GTEX-1128S-2726-SM-5H12C,4.580119,45.050399,2.099978e-01,-1.089298e-08,-1.089298e-08,6.000199e-02,-1.089298e-08,9.829740,26.430749,1.570018
5,gtex_brain,GTEX-1128S-2826-SM-5N9DI,9.870026,56.908595,3.499896e-01,5.600106e-01,-1.089298e-08,-1.089298e-08,-1.089298e-08,6.659821,38.718811,2.140023
...,...,...,...,...,...,...,...,...,...,...,...,...
1297,gbm,TCGA-76-4927-01,1.099989,164.882131,2.999926e-01,-1.089298e-08,5.238103e+01,2.399992e-01,5.999846e-01,48.778396,111.816331,2.479963
1298,gbm,TCGA-76-4928-01,2.230015,115.303100,1.700038e-01,1.423034e+01,7.899728e-01,5.200162e-01,-1.089298e-08,36.969432,75.138520,1.379987
1299,gbm,TCGA-76-4929-01,3.530057,149.239987,2.199972e-01,1.046980e+01,3.088969e+01,2.599961e-01,-1.089298e-08,51.681358,91.721260,2.559987
1300,gbm,TCGA-76-4931-01,8.330060,163.993088,-1.089298e-08,9.230084e+00,5.668812e+01,2.399992e-01,4.200023e-01,57.759047,85.780819,4.050062


In [6]:
print(df.groupby(['status']).size())
groups_name = list(df.groupby(['status']).groups.keys())
print(groups_name)

status
gbm            153
gtex_brain    1148
dtype: int64
['gbm', 'gtex_brain']


In [7]:
def stratified_sample(df, strata, size=None, seed=None, keep_index= True):
    population = len(df)
    #print(population, size)
    size = __smpl_size(population, size)
    tmp = df.loc[:,strata]
    tmp['size'] = 1
    tmp_grpd = tmp.groupby(strata).count().reset_index()

    tmp_grpd['samp_size'] = np.round(size/population * tmp_grpd['size']).astype(int)

    # controlling variable to create the dataframe or append to it
    first = True 
    for i in range(len(tmp_grpd)):
        # query generator for each iteration
        qry=''
        for s in range(len(strata)):
            stratum = strata[s]
            value = tmp_grpd.iloc[i][stratum]
            n = tmp_grpd.iloc[i]['samp_size']

            if type(value) == str:
                value = "'" + str(value) + "'"
            
            if s != len(strata)-1:
                qry = qry + stratum + ' == ' + str(value) +' & '
            else:
                qry = qry + stratum + ' == ' + str(value)
        
        # final dataframe
        if first:
            stratified_df = df.query(qry).sample(n=n, random_state=seed).reset_index(drop=(not keep_index))
            first = False
        else:
            tmp_df = df.query(qry).sample(n=n, random_state=seed).reset_index(drop=(not keep_index))
            stratified_df = stratified_df.append(tmp_df, ignore_index=True)
    
    return stratified_df

In [8]:
def __smpl_size(population, size):
    '''
    A function to compute the sample size. If not informed, a sampling 
    size will be calculated using Cochran adjusted sampling formula:
        cochran_n = (Z**2 * p * q) /e**2
        where:
            - Z is the z-value. In this case we use 1.96 representing 95%
            - p is the estimated proportion of the population which has an
                attribute. In this case we use 0.5
            - q is 1-p
            - e is the margin of error
        This formula is adjusted as follows:
        adjusted_cochran = cochran_n / 1+((cochran_n -1)/N)
        where:
            - cochran_n = result of the previous formula
            - N is the population size
    '''
    if size is None:
        cochran_n = np.round(((1.96)**2 * 0.5 * 0.5)/ 0.02**2)
        n = np.round(cochran_n/(1+((cochran_n -1) /population)))
    elif size >= 0 and size < 1:
        n = np.round(population * size)
    elif size < 0:
        raise ValueError('Parameter "size" must be an integer or a proportion between 0 and 0.99.')
    elif size >= 1:
        n = size
    return n

In [9]:
temp_list = []
for i in range(no_of_iteration):
    temp_stratified = stratified_sample(df=df, strata=['status'], size=0.8)
    print(temp_stratified.groupby(['status']).size())
    temp_list.append(temp_stratified)

status
gbm           122
gtex_brain    919
dtype: int64
status
gbm           122
gtex_brain    919
dtype: int64


/tmp/ipykernel_458965/2322919580.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stratified_df = stratified_df.append(tmp_df, ignore_index=True)
/tmp/ipykernel_458965/2322919580.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  stratified_df = stratified_df.append(tmp_df, ignore_index=True)


In [87]:
temp_list[0].dtypes

index                  int64
status                object
sample                object
ENST00000461347.5    float64
ENST00000322723.8    float64
ENST00000484328.1    float64
ENST00000454824.5    float64
ENST00000417652.5    float64
ENST00000453992.5    float64
ENST00000436894.1    float64
ENST00000494618.1    float64
ENST00000356936.5    float64
ENST00000466274.1    float64
dtype: object

In [88]:
df_p_values = pd.DataFrame()
df_p_values['Transcript_ID'] = df.columns[2:]
iteration =0
for df_temp in temp_list:
    iteration +=1
    if (iteration% 25) ==0:
        print(iteration)
    #print(df_temp.groupby(['lung_tissue']).size())                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
    cat1 = df_temp[df_temp['status']==groups_name[0]]
    cat2 = df_temp[df_temp['status']==groups_name[1]]
    for index, row in df_p_values.iterrows():
        tuple_values = ttest_ind(cat1[row['Transcript_ID']], cat2[row['Transcript_ID']])
        df_p_values.loc[index, ['ITER_{}'.format(iteration)]]= tuple_values[1]
        index+=1

25
50
75
100
125
150
175
200
225
250
275
300
325
350
375
400
425
450
475
500


In [89]:
df_p_values.set_index('Transcript_ID', inplace=True)
df_p_values

,ITER_1,ITER_2,ITER_3,ITER_4,ITER_5,ITER_6,ITER_7,ITER_8,ITER_9,ITER_10,...,ITER_491,ITER_492,ITER_493,ITER_494,ITER_495,ITER_496,ITER_497,ITER_498,ITER_499,ITER_500
Transcript_ID,,,,,,,,,,,,,,,,,,,,,
ENST00000461347.5,4.757755e-04,6.602261e-04,2.283010e-04,7.697847e-04,2.189386e-04,5.794817e-04,2.795030e-04,9.058486e-04,6.959711e-04,2.787369e-05,...,1.083154e-03,4.180765e-04,9.490107e-04,1.220877e-03,1.741856e-03,2.679394e-04,1.654097e-05,8.846006e-04,2.402125e-04,4.693024e-04
ENST00000322723.8,7.451148e-71,9.762586e-77,7.533849e-67,1.504272e-69,2.008977e-65,2.246975e-65,2.303743e-74,5.287413e-66,1.474002e-71,8.188716e-65,...,1.543877e-76,1.773959e-67,5.482935e-67,9.772857e-66,1.107639e-72,1.223206e-67,2.447860e-62,5.033376e-74,2.160240e-73,1.368365e-67
ENST00000484328.1,3.179928e-01,6.540731e-01,2.177607e-01,5.096610e-01,3.299424e-01,1.809601e-01,6.180199e-01,9.756402e-01,4.201971e-01,4.499042e-01,...,3.423356e-01,4.445751e-01,4.284728e-01,3.098008e-01,5.928011e-01,6.346227e-01,4.238441e-01,4.085438e-01,2.416185e-01,1.672413e-01
ENST00000454824.5,1.595569e-78,2.435926e-75,5.944072e-72,1.366732e-85,9.163744e-73,7.267856e-82,9.316994e-74,2.562087e-76,2.932698e-72,1.020440e-81,...,3.037705e-71,9.825973e-67,2.215771e-68,4.785316e-85,1.089928e-75,4.955389e-74,1.269064e-73,3.942312e-76,1.209193e-74,5.998863e-71
ENST00000417652.5,4.979809e-46,3.527072e-48,3.607827e-43,3.342301e-40,5.524339e-42,4.116821e-40,1.893738e-52,4.124854e-44,5.717352e-45,6.825278e-40,...,4.941115e-40,4.402474e-42,9.406800e-43,5.643606e-39,1.033895e-44,1.758543e-49,5.098331e-41,2.859298e-47,4.924457e-44,2.278518e-37
ENST00000453992.5,6.060134e-01,2.185710e-01,2.568939e-01,3.667364e-01,5.147923e-01,3.596022e-01,1.812144e-01,4.232513e-01,2.271109e-01,3.443907e-01,...,2.807960e-01,4.719033e-01,3.812323e-01,2.099323e-01,2.196771e-01,4.445940e-01,3.185335e-01,2.009284e-01,3.339271e-01,4.059998e-01
ENST00000436894.1,5.003744e-01,8.531856e-01,4.153735e-01,2.453712e-01,4.280321e-01,7.826311e-01,6.349164e-02,5.543866e-01,2.642060e-01,7.104840e-01,...,4.855741e-01,6.210365e-01,3.525429e-01,5.379542e-01,5.570956e-01,9.074275e-01,9.633392e-01,3.093838e-01,1.945834e-01,5.014458e-01
ENST00000494618.1,1.110799e-43,2.102075e-42,5.397847e-41,5.674421e-39,9.477496e-34,5.843564e-40,5.229855e-49,3.565389e-41,2.428080e-41,1.255354e-37,...,6.082373e-43,1.109619e-39,2.493931e-38,1.303944e-37,4.968629e-46,1.774986e-39,2.668227e-37,2.522121e-46,2.183346e-43,3.217253e-38
ENST00000356936.5,4.999457e-123,2.742490e-125,1.565035e-119,5.933763e-123,3.067124e-115,5.527126e-114,5.223940e-122,1.560463e-118,2.183264e-121,2.243048e-114,...,3.063784e-129,2.166796e-121,3.083958e-119,1.388791e-119,2.951200e-122,1.311288e-111,3.656854e-110,3.027616e-125,2.874336e-124,2.566061e-118


In [90]:
df_p_values.to_csv(OUTPUT_PATH+"/"+INPUT_FOLDERNAME+"/"+OUTPUT_FILE_NAME, sep=",")

In [91]:
def count(df):
    #n=df.shape[1]
    #print(df)
    threshold_values=[0.05]
    for threshold in threshold_values:
        r = np.sum(df >= threshold)
        df['>={}'.format(threshold)] = (r+1) /(no_of_iteration+1)
    #df['r'] = np.sum(df == 'r')
    return df

In [92]:
temp_df_empirical = df_p_values.apply(count, axis = 1)

In [93]:
temp_df_empirical.iloc[: , -1:]

,>=0.05
Transcript_ID,
ENST00000461347.5,0.001996
ENST00000322723.8,0.001996
ENST00000484328.1,1.000000
ENST00000454824.5,0.001996
ENST00000417652.5,0.001996
ENST00000453992.5,0.998004
ENST00000436894.1,0.994012
ENST00000494618.1,0.001996
ENST00000356936.5,0.001996


In [94]:
df_mean = df.groupby("status").mean(numeric_only=True)
df_mean

,ENST00000461347.5,ENST00000322723.8,ENST00000484328.1,ENST00000454824.5,ENST00000417652.5,ENST00000453992.5,ENST00000436894.1,ENST00000494618.1,ENST00000356936.5,ENST00000466274.1
status,,,,,,,,,,
gbm,2.900284,117.152197,0.090392,10.418378,21.412329,0.171307,0.046667,32.872318,70.108631,2.412982
gtex_brain,4.432685,61.832839,0.110166,2.182572,4.562927,0.152230,0.031786,14.599218,29.145744,2.512778


In [95]:
index_list = list(df_mean.index.values)

In [96]:
df_mean.loc['Fold_change_'+index_list[0]+'/'+index_list[1]] = df_mean.iloc[0]/df_mean.iloc[1]
df_mean.loc['Fold_change_'+index_list[1]+'/'+index_list[0]] = df_mean.iloc[1]/df_mean.iloc[0]
df_mean

,ENST00000461347.5,ENST00000322723.8,ENST00000484328.1,ENST00000454824.5,ENST00000417652.5,ENST00000453992.5,ENST00000436894.1,ENST00000494618.1,ENST00000356936.5,ENST00000466274.1
status,,,,,,,,,,
gbm,2.900284,117.152197,0.090392,10.418378,21.412329,0.171307,0.046667,32.872318,70.108631,2.412982
gtex_brain,4.432685,61.832839,0.110166,2.182572,4.562927,0.152230,0.031786,14.599218,29.145744,2.512778
Fold_change_gbm/gtex_brain,0.654295,1.894660,0.820509,4.773442,4.692674,1.125317,1.468161,2.251649,2.405450,0.960285
Fold_change_gtex_brain/gbm,1.528362,0.527799,1.218755,0.209492,0.213098,0.888638,0.681124,0.444119,0.415723,1.041358


In [99]:
temp_df_empirical['Fold_change_gbm/gtex_brain'] = df_mean.loc['Fold_change_gbm/gtex_brain'].values[:]
temp_df_empirical['Fold_change_gtex_brain/gbm'] = df_mean.loc['Fold_change_gtex_brain/gbm'].values[:]

In [100]:
temp_df_empirical

,ITER_1,ITER_2,ITER_3,ITER_4,ITER_5,ITER_6,ITER_7,ITER_8,ITER_9,ITER_10,...,ITER_494,ITER_495,ITER_496,ITER_497,ITER_498,ITER_499,ITER_500,>=0.05,Fold_change_gbm/gtex_brain,Fold_change_gtex_brain/gbm
Transcript_ID,,,,,,,,,,,,,,,,,,,,,
ENST00000461347.5,4.757755e-04,6.602261e-04,2.283010e-04,7.697847e-04,2.189386e-04,5.794817e-04,2.795030e-04,9.058486e-04,6.959711e-04,2.787369e-05,...,1.220877e-03,1.741856e-03,2.679394e-04,1.654097e-05,8.846006e-04,2.402125e-04,4.693024e-04,0.001996,0.654295,1.528362
ENST00000322723.8,7.451148e-71,9.762586e-77,7.533849e-67,1.504272e-69,2.008977e-65,2.246975e-65,2.303743e-74,5.287413e-66,1.474002e-71,8.188716e-65,...,9.772857e-66,1.107639e-72,1.223206e-67,2.447860e-62,5.033376e-74,2.160240e-73,1.368365e-67,0.001996,1.894660,0.527799
ENST00000484328.1,3.179928e-01,6.540731e-01,2.177607e-01,5.096610e-01,3.299424e-01,1.809601e-01,6.180199e-01,9.756402e-01,4.201971e-01,4.499042e-01,...,3.098008e-01,5.928011e-01,6.346227e-01,4.238441e-01,4.085438e-01,2.416185e-01,1.672413e-01,1.000000,0.820509,1.218755
ENST00000454824.5,1.595569e-78,2.435926e-75,5.944072e-72,1.366732e-85,9.163744e-73,7.267856e-82,9.316994e-74,2.562087e-76,2.932698e-72,1.020440e-81,...,4.785316e-85,1.089928e-75,4.955389e-74,1.269064e-73,3.942312e-76,1.209193e-74,5.998863e-71,0.001996,4.773442,0.209492
ENST00000417652.5,4.979809e-46,3.527072e-48,3.607827e-43,3.342301e-40,5.524339e-42,4.116821e-40,1.893738e-52,4.124854e-44,5.717352e-45,6.825278e-40,...,5.643606e-39,1.033895e-44,1.758543e-49,5.098331e-41,2.859298e-47,4.924457e-44,2.278518e-37,0.001996,4.692674,0.213098
ENST00000453992.5,6.060134e-01,2.185710e-01,2.568939e-01,3.667364e-01,5.147923e-01,3.596022e-01,1.812144e-01,4.232513e-01,2.271109e-01,3.443907e-01,...,2.099323e-01,2.196771e-01,4.445940e-01,3.185335e-01,2.009284e-01,3.339271e-01,4.059998e-01,0.998004,1.125317,0.888638
ENST00000436894.1,5.003744e-01,8.531856e-01,4.153735e-01,2.453712e-01,4.280321e-01,7.826311e-01,6.349164e-02,5.543866e-01,2.642060e-01,7.104840e-01,...,5.379542e-01,5.570956e-01,9.074275e-01,9.633392e-01,3.093838e-01,1.945834e-01,5.014458e-01,0.994012,1.468161,0.681124
ENST00000494618.1,1.110799e-43,2.102075e-42,5.397847e-41,5.674421e-39,9.477496e-34,5.843564e-40,5.229855e-49,3.565389e-41,2.428080e-41,1.255354e-37,...,1.303944e-37,4.968629e-46,1.774986e-39,2.668227e-37,2.522121e-46,2.183346e-43,3.217253e-38,0.001996,2.251649,0.444119
ENST00000356936.5,4.999457e-123,2.742490e-125,1.565035e-119,5.933763e-123,3.067124e-115,5.527126e-114,5.223940e-122,1.560463e-118,2.183264e-121,2.243048e-114,...,1.388791e-119,2.951200e-122,1.311288e-111,3.656854e-110,3.027616e-125,2.874336e-124,2.566061e-118,0.001996,2.405450,0.415723


In [101]:
temp_df_empirical.iloc[: , -3:]

,>=0.05,Fold_change_gbm/gtex_brain,Fold_change_gtex_brain/gbm
Transcript_ID,,,
ENST00000461347.5,0.001996,0.654295,1.528362
ENST00000322723.8,0.001996,1.894660,0.527799
ENST00000484328.1,1.000000,0.820509,1.218755
ENST00000454824.5,0.001996,4.773442,0.209492
ENST00000417652.5,0.001996,4.692674,0.213098
ENST00000453992.5,0.998004,1.125317,0.888638
ENST00000436894.1,0.994012,1.468161,0.681124
ENST00000494618.1,0.001996,2.251649,0.444119
ENST00000356936.5,0.001996,2.405450,0.415723


In [102]:
temp_df_empirical.iloc[: , -3:].to_csv(OUTPUT_PATH+"/"+INPUT_FOLDERNAME+"/"+OUTPUT_FILE_NAME1, sep=",")